In [1]:
# 网络摄像头代码，用于采集数据（图像）
# 代码编写：谢作如(2021.4.12)

import remi.gui as gui
from remi import start, App
import time,threading
import qrcode,base64,io,cv2
import gradio as gr
import BaseDeploy as bd

model_path = 'E:\\Lindia\\2023.08.21 AI Lab 夏令营\\视力检测\\out_file\\hand.onnx'
model = bd(model_path)

camera = cv2.VideoCapture(0)
# 临时变量，用于保存画面
tempcam = None
# 图片名称编号
f_num = 1

# # 二维码信息提示
# def get_img(data):
#     img = qrcode.make(data)
#     out = io.BytesIO()
#     img.save(out, 'PNG')
#     base64_data=base64.b64encode(out.getvalue())
#     s=base64_data.decode()
#     data='data:image/jpeg;base64,%s'%s
#     return data

# 返回摄像头画面，自定义大小
def get_frames(x,y):
    global tempcam
    success, frame = camera.read()  # read the camera frame
    if success:
        tempcam = frame #临时变量获取摄像头画面
        frame = cv2.flip(frame, 1, dst=None) #镜像
        frame = cv2.resize(frame, (x, y), interpolation=cv2.INTER_LINEAR) #自定义大小
        image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
        base64_data = str(base64.b64encode(image))[2:-1]
        data='data:image/jpeg;base64,%s'%base64_data
        return data
    else:
        return get_img("摄像头启动失败")

# Web主程序
class MyApp(App):
    def __init__(self, *args):
        super(MyApp, self).__init__(*args)

    def main(self):
        #页面大小
        container = gui.VBox(width=700, height=450,style={'margin':'0px auto'}) 
        
        #按钮所在的框
        bts = gui.HBox(width=300, height=50)
        
        #页面元素
        self.lbl_01 = gui.Label('Web摄像头画面采集器',style={'font-size': '25px'})
        self.lbl_9 = gui.Label(' ')
        self.img_1 = gui.Image(get_frames(200,100))
        self.bt_cap = gui.Button("Capture") #拍照键
        self.photo = gui.Image(get_frames(200,100)) #显示拍摄照片的框
        self.result = gui.Label('Result will show here..')
        self.photo= gui.Image(get_frames(200,100))

        # 按钮按下时执行
        self.bt_cap.onclick.do(self.on_button_pressed,1,container)

        # 添加到网页上
        container.append(self.lbl_01)
        container.append(self.lbl_9)
        container.append(self.img_1)
        container.append(self.photo)
        container.append(self.result)
        

        bts.append(self.bt_cap)
        container.append(bts)
        
        # 开启新的进程刷新画面
        t = threading.Thread(target=self.showimg)
        t.start()

        # returning the root widget
        return container

    def on_button_pressed(self, emitter, n, container):
        print(n)
        if n==1: #拍照并预测
            test=tempcam
            frame = tempcam
            frame = cv2.flip(frame, 1, dst=None) #镜像
            frame = cv2.resize(frame, (200,100), interpolation=cv2.INTER_LINEAR) #自定义大小
            image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
            base64_data = str(base64.b64encode(image))[2:-1]
            data='data:image/jpeg;base64,%s'%base64_data
            self.photo.set_image(data)
            result = model.inference(test)
            result = model.print_result(result)
            print(result)
            self.result.set_text(result['预测结果'])
            
            

        if n==2:
            f_data = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime())
            cv2.imwrite(str(f_data) + '.jpg',tempcam)
    # 刷新网页的摄像头画面
    def showimg(self):
        while True:
            self.img_1.set_image(get_frames(200,100))
            time.sleep(0.2)

# starts the web server
start(MyApp,title='摄像头实时视频',address='0.0.0.0')


Success load model info generate by MMEdu>=0.1.15: {"codebase": "MMCls", "modelname": "LeNet", "classes": ["down", "left", "right", "up"]}


remi.server      INFO     Started httpserver http://0.0.0.0:64254/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [25/Aug/2023 10:15:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2023 10:15:32] "GET /res:style.css HTTP/1.1" 200 -
remi.server.ws   INFO     connection established: ('127.0.0.1', 64257)
remi.server.ws   INFO     handshake complete
127.0.0.1 - - [25/Aug/2023 10:15:32] "GET /res:font.woff2 HTTP/1.1" 200 -


1
{'标签': 1, '置信度': 0.6472183, '预测结果': 'left'}
1
{'标签': 1, '置信度': 0.59531945, '预测结果': 'left'}
1
{'标签': 1, '置信度': 0.93836725, '预测结果': 'left'}
1
{'标签': 0, '置信度': 0.5510175, '预测结果': 'down'}


remi.server      INFO     *** signal 2 received.
